In [1]:
import pandas as pd
import numpy as np
import random
random.seed(10)
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn import neighbors
from sklearn.model_selection import train_test_split, GridSearchCV, RepeatedStratifiedKFold
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier, KernelDensity
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.utils import resample
from scipy.stats import friedmanchisquare
import scikit_posthocs as sp

In [2]:
# Importando as 3 bases de dados
df_fac = pd.read_csv('C:/Users/terti/jupyter_lab/AM - Chico/multiple+features/mfeat-fac', header=None, sep='\s+')
df_fou = pd.read_csv('C:/Users/terti/jupyter_lab/AM - Chico/multiple+features/mfeat-fou', header=None, sep='\s+')
df_zer = pd.read_csv('C:/Users/terti/jupyter_lab/AM - Chico/multiple+features/mfeat-zer', header=None, sep='\s+')

# Data exploratory

In [3]:
df_fac.head(5)

,0,1,2,3,4,5,6,7,8,9,...,206,207,208,209,210,211,212,213,214,215
0,98,236,531,673,607,647,2,9,3,6,...,474,536,628,632,18,36,8,15,12,13
1,121,193,607,611,585,665,7,9,2,4,...,520,458,570,634,15,32,11,13,15,11
2,115,141,590,605,557,627,12,6,3,3,...,535,498,572,656,20,35,16,14,13,6
3,90,122,627,692,607,642,0,6,4,5,...,576,549,628,621,16,35,7,12,15,9
4,157,167,681,666,587,666,8,6,1,4,...,594,525,568,653,16,35,10,15,13,13


In [4]:
df_fou.head(5)

,0,1,2,3,4,5,6,7,8,9,...,66,67,68,69,70,71,72,73,74,75
0,0.065882,0.197312,0.103826,0.270362,0.616078,0.035856,0.424572,0.089701,0.367773,0.037065,...,0.123448,0.113374,0.074343,0.281666,0.067795,0.344182,0.038963,0.394366,0.049971,0.344871
1,0.049142,0.175971,0.105515,0.227095,0.599280,0.041217,0.431078,0.096801,0.326739,0.059661,...,0.045310,0.069337,0.045386,0.254264,0.045447,0.335659,0.021719,0.445277,0.083978,0.354092
2,0.034172,0.227649,0.108766,0.127697,0.612494,0.056554,0.470639,0.041903,0.324267,0.044569,...,0.019858,0.218842,0.041087,0.360464,0.047154,0.377408,0.052099,0.445029,0.071234,0.261465
3,0.062336,0.217979,0.080243,0.289592,0.546316,0.045779,0.425545,0.022841,0.331454,0.119052,...,0.083995,0.085479,0.087658,0.151880,0.075950,0.293462,0.022675,0.408291,0.063010,0.401376
4,0.061970,0.198358,0.111239,0.253460,0.608455,0.023631,0.415246,0.091866,0.309310,0.049142,...,0.021004,0.106960,0.032283,0.248565,0.015674,0.386276,0.039481,0.434701,0.069218,0.405403


In [5]:
df_zer.head(5)

,0,1,2,3,4,5,6,7,8,9,...,37,38,39,40,41,42,43,44,45,46
0,0.011033,0.831466,15.351804,75.806559,171.554214,490.156556,206.416027,0.122135,2.601646,11.472709,...,1.713529,33.810340,9.858915,1.399891,148.138058,326.239452,9.711070,20.007248,47.032578,539.208457
1,0.038271,1.166746,10.526913,42.369276,85.187116,420.360566,253.569574,0.033657,0.390566,11.700830,...,2.590208,35.400531,70.681899,6.674412,155.135985,377.832675,8.140633,44.536711,46.338954,518.496567
2,0.042698,1.225007,8.273804,31.744786,54.448177,404.103204,389.980746,0.041733,0.937399,11.629045,...,1.476285,19.477230,30.093590,7.858211,150.126419,419.565747,4.530921,26.292170,44.574822,549.912691
3,0.032418,1.638247,19.205283,51.196682,57.181760,429.052011,256.174645,0.073624,1.973268,13.057108,...,1.349613,14.179518,30.564085,7.097728,173.840759,441.350376,3.706023,13.432311,51.739930,574.887814
4,0.015866,0.611561,8.627839,37.325052,48.509025,459.909634,238.572767,0.046477,1.117292,10.012169,...,0.667971,8.705403,30.242473,9.015714,167.021185,332.479997,1.806273,23.689300,50.407770,492.227513


In [6]:
df_fac.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Columns: 216 entries, 0 to 215
dtypes: int64(216)
memory usage: 3.3 MB


In [7]:
df_fou.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 76 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       2000 non-null   float64
 1   1       2000 non-null   float64
 2   2       2000 non-null   float64
 3   3       2000 non-null   float64
 4   4       2000 non-null   float64
 5   5       2000 non-null   float64
 6   6       2000 non-null   float64
 7   7       2000 non-null   float64
 8   8       2000 non-null   float64
 9   9       2000 non-null   float64
 10  10      2000 non-null   float64
 11  11      2000 non-null   float64
 12  12      2000 non-null   float64
 13  13      2000 non-null   float64
 14  14      2000 non-null   float64
 15  15      2000 non-null   float64
 16  16      2000 non-null   float64
 17  17      2000 non-null   float64
 18  18      2000 non-null   float64
 19  19      2000 non-null   float64
 20  20      2000 non-null   float64
 21  21      2000 non-null   float64
 22  

In [8]:
df_zer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 47 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       2000 non-null   float64
 1   1       2000 non-null   float64
 2   2       2000 non-null   float64
 3   3       2000 non-null   float64
 4   4       2000 non-null   float64
 5   5       2000 non-null   float64
 6   6       2000 non-null   float64
 7   7       2000 non-null   float64
 8   8       2000 non-null   float64
 9   9       2000 non-null   float64
 10  10      2000 non-null   float64
 11  11      2000 non-null   float64
 12  12      2000 non-null   float64
 13  13      2000 non-null   float64
 14  14      2000 non-null   float64
 15  15      2000 non-null   float64
 16  16      2000 non-null   float64
 17  17      2000 non-null   float64
 18  18      2000 non-null   float64
 19  19      2000 non-null   float64
 20  20      2000 non-null   float64
 21  21      2000 non-null   float64
 22  



As we have seen in the dataset link: "Multiple features" fromthe website uci machine learning repository
(https://archive.ics.uci.edu/dataset/72/multiple+features).

In each file the 2000 patterns are stored in ASCI on 2000 lines. The first 200 patterns are of class `0', followed by sets of 200 patterns for each of the classes `1' - `9'. Corresponding patterns in different feature sets (files) correspond to the same original character.

We will create the target column in a separate dataframe

In [9]:
df_target = pd.DataFrame(np.repeat(np.arange(10), 200))
df_target.columns = ["target"]
df_target.shape

(2000, 1)

In [10]:
#taking a look at some random rows from the new target column
df_target.sample(n=10)

,target
1277,6
1639,8
1745,8
1727,8
881,4
1874,9
80,0
1324,6
1620,8
1199,5


In [11]:
# verifying NaN values
df_fac.isna().sum()


0      0
1      0
2      0
3      0
4      0
      ..
211    0
212    0
213    0
214    0
215    0
Length: 216, dtype: int64

In [12]:
df_fou.isna().sum()

0     0
1     0
2     0
3     0
4     0
     ..
71    0
72    0
73    0
74    0
75    0
Length: 76, dtype: int64

In [13]:
df_zer.isna().sum()

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
20    0
21    0
22    0
23    0
24    0
25    0
26    0
27    0
28    0
29    0
30    0
31    0
32    0
33    0
34    0
35    0
36    0
37    0
38    0
39    0
40    0
41    0
42    0
43    0
44    0
45    0
46    0
dtype: int64

# Questão 2 parte 1 
Alternativa A)
Use validação cruzada estratificada “30 × 10 folds” para avaliar e comparar os 4 classificadores usando a regra do voto majoritário baseados, respectivamente, nos classificadores bayesiano gaussiano, bayesiano baseado em k-vizinhos, bayesiano baseado na janela de Parzen e regressão logística. Quando necessario, retire do conjunto de aprendizagem, um conjunto de validação (20%) para fazer ajuste de hiper-parametros e depois treine o modelo novamente com o conjunto aprendizagem + validação. Use amostragem estratificada

### Dataset mfeat-fac

In [14]:
# data split
X_train_fac, X_test_fac, y_train_fac, y_test_fac = train_test_split(df_fac,
                                                                    df_target,
                                                                    test_size=0.3,          # Using 30% of the dataset for test
                                                                    stratify= df_target,    # Stratified sample
                                                                    random_state=10)
# Standard Scaler
scaler = StandardScaler()
scaler.fit(X_train_fac)

X_train_fac_scaled = scaler.transform(X_train_fac)
X_test_fac_scaled = scaler.transform(X_test_fac)
# Before fitting the model, convert y_train_fac to a 1D array
y_train_fac = y_train_fac.values.ravel()
y_test_fac = y_test_fac.values.ravel()


In [15]:
# (30x10) stratified fold
cv = RepeatedStratifiedKFold(n_splits = 10, n_repeats = 30, random_state = 10)

In [16]:
# Classifier KNN
knn = neighbors.KNeighborsClassifier(weights='distance', metric='euclidean')
param_grid_KNN = param_grid = {
    'n_neighbors': range(1, 20)
}
grid_KNN_fac = GridSearchCV(knn, param_grid = param_grid_KNN, cv=cv, refit='f1_macro')         
grid_KNN_fac.fit(X_train_fac_scaled, y_train_fac)                                                                                           
best_KNN_fac = grid_KNN_fac.best_params_                                                                                  
print("Best fac-KNN parameter:", best_KNN_fac)                                                                           
best_clf_KNN_fac = KNeighborsClassifier(n_neighbors = best_KNN_fac.get('n_neighbors'), weights = 'distance', metric = 'euclidean')
best_clf_KNN_fac.fit(X_train_fac_scaled, y_train_fac)

Best fac-KNN parameter: {'n_neighbors': 5}


KNeighborsClassifier(metric='euclidean', weights='distance')

In [17]:
# Classifier Gaussian Bayesian
clf_GNB_fac = GaussianNB()   # there are no hyperparameters
clf_GNB_fac.fit(X_train_fac_scaled, y_train_fac) 

GaussianNB()

In [18]:
# Classifier Parzen Window
param_grid_parzen_window = {'bandwidth': np.linspace(0.01, 1, 20)}

parzen_window = KernelDensity(kernel='gaussian')
grid_parzen_window = GridSearchCV(parzen_window, param_grid_parzen_window, cv=cv, refit='f1_macro')
grid_parzen_window.fit(X_train_fac_scaled, y_train_fac)
best_parzen_window = grid_parzen_window.best_params_
print("Best fac parameter:", best_parzen_window)
best_clf_parzen_window_fac = KernelDensity(kernel='gaussian', bandwidth = best_parzen_window.get('bandwidth'))
best_clf_parzen_window_fac.fit(X_train_fac_scaled, y_train_fac)

class Parzen_Window(BaseEstimator, ClassifierMixin):
    def __init__(self, bandwidth=1.0):
        self.bandwidth = bandwidth
        self.kde = None
        self.classes_ = None

    def fit(self, X, y):
        self.classes_ = np.unique(y)
        self.kde = {cls: KernelDensity(bandwidth=self.bandwidth).fit(X[y == cls]) for cls in self.classes_}
        return self

    def predict(self, X):
        log_probs = np.array([self.kde[cls].score_samples(X) for cls in self.classes_]).T
        return self.classes_[np.argmax(log_probs, axis=1)]

best_clf_parzen_window_fac = Parzen_Window(bandwidth=best_parzen_window.get('bandwidth'))
best_clf_parzen_window_fac.fit(X_train_fac_scaled, y_train_fac)

Best fac parameter: {'bandwidth': 0.32263157894736844}


Parzen_Window(bandwidth=0.32263157894736844)

In [19]:
# Logistic Regression
param_grid_LR = {'C':  np.linspace(0.01, 10, 20), 'random_state': np.arange(0, 100, 5)}
grid_LR_fac = GridSearchCV(LogisticRegression(max_iter=1000), param_grid = param_grid_LR, cv=cv, refit='f1_macro')                                                    
grid_LR_fac.fit(X_train_fac_scaled, y_train_fac)                                                                                              
best_LR_fac = grid_LR_fac.best_params_
print("Best fac-LR parameter:", best_LR_fac)
best_clf_LR_fac = LogisticRegression(C = best_LR_fac.get('C'), random_state = best_LR_fac.get('random_state'), max_iter = 1000)
best_clf_LR_fac.fit(X_train_fac_scaled, y_train_fac)

Best fac-LR parameter: {'C': 1.0615789473684212, 'random_state': 0}


LogisticRegression(C=1.0615789473684212, max_iter=1000, random_state=0)

In [20]:
# Voting classifier (majority rule)
classifiers = [
    ('knn', best_clf_KNN_fac),
    ('gnb', clf_GNB_fac),
    ('parzen', best_clf_parzen_window_fac),
    ('lr', best_clf_LR_fac)
]
voting_clf = VotingClassifier(estimators=classifiers, voting='hard')
voting_clf.fit(X_train_fac_scaled, y_train_fac)

# accuracy on train test
train_accuracy = voting_clf.score(X_train_fac_scaled, y_train_fac)
print("Precisão no conjunto de treinamento:", train_accuracy)

# accuracy on test set
test_accuracy = voting_clf.score(X_test_fac_scaled, y_test_fac)
print("Precisão no conjunto de teste:", test_accuracy)


Precisão no conjunto de treinamento: 1.0
Precisão no conjunto de teste: 0.965


# Alternativa B
Obtenha uma estimativa pontual e um intervalo de confiança para cada metrica
de avaliação do classificador (Taxa de erro, precisão, cobertura, F-measure);

In [21]:
y_pred = voting_clf.predict(X_test_fac_scaled)

error_rate = 1 - voting_clf.score(X_test_fac_scaled, y_test_fac)
print("error rate:", error_rate)

coverage_rate = recall_score(y_test_fac, y_pred, average='macro')
print("Recall:", coverage_rate)

precision = precision_score(y_test_fac, y_pred, average='macro')
print("Precision:", precision)

f1 = f1_score(y_test_fac, y_pred, average='macro')
print("F1-score:", f1)

error rate: 0.03500000000000003
Recall: 0.9649999999999999
Precision: 0.9652144929916744
F1-score: 0.9647732351327292


In [22]:
# Function to calculate each metric
def calculate_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred, average='macro')
    precision = precision_score(y_true, y_pred, average='macro')
    f1 = f1_score(y_true, y_pred, average='macro')
    error_rate = 1 - accuracy
    return accuracy, recall, precision, f1, error_rate

# bootstrap iterations number
n_iterations = 1000
# array for result storage
metrics_results = np.zeros((n_iterations, 5))

# running the bootstrap
for i in range(n_iterations):
    # Search for a sample with reposition
    X_resampled, y_resampled = resample(X_train_fac_scaled, y_train_fac)
    # training the model with the sample
    voting_clf.fit(X_resampled, y_resampled)
    # making predictions
    y_pred_resampled = voting_clf.predict(X_resampled)
    # calculating metrics
    metrics_results[i] = calculate_metrics(y_resampled, y_pred_resampled)

# calculating the confidence interval for each metric
alpha = 0.95
lower_p = ((1.0 - alpha) / 2.0) * 100
upper_p = (alpha + ((1.0 - alpha) / 2.0)) * 100
intervals = np.percentile(metrics_results, [lower_p, upper_p], axis=0)

# Imprimindo os intervalos de confiança
metrics_names = ['accuracy', 'Recall', 'Precision', 'F1-score', 'error-rate']
for metric_index, metric_name in enumerate(metrics_names):
    print(f"{metric_name} Confidence interval {alpha*100}%: Lower_p: {intervals[0][metric_index]:.2f}%, Upper_p: {intervals[1][metric_index]:.2f}%")


accuracy Confidence interval 95.0%: Lower_p: 1.00%, Upper_p: 1.00%
Recall Confidence interval 95.0%: Lower_p: 1.00%, Upper_p: 1.00%
Precision Confidence interval 95.0%: Lower_p: 1.00%, Upper_p: 1.00%
F1-score Confidence interval 95.0%: Lower_p: 1.00%, Upper_p: 1.00%
error-rate Confidence interval 95.0%: Lower_p: 0.00%, Upper_p: 0.00%


In [23]:
scorers = {
    'accuracy': make_scorer(accuracy_score),
    'recall_macro': make_scorer(recall_score, average='macro'),
    'f1_macro': make_scorer(f1_score, average='macro'),
    'precision_macro': make_scorer(precision_score, average='macro', zero_division = 0),
}

In [24]:
KNN_cv_results = cross_validate(best_clf_KNN_fac, X_train_fac_scaled, y_train_fac, cv=cv, scoring=scorers)

In [25]:
GNB_cv_results = cross_validate(clf_GNB_fac, X_train_fac_scaled, y_train_fac, cv=cv, scoring=scorers)

In [26]:
PW_cv_results = cross_validate(best_clf_parzen_window_fac, X_train_fac_scaled, y_train_fac, cv=cv, scoring=scorers)

In [27]:
LR_cv_results = cross_validate(best_clf_LR_fac, X_train_fac_scaled, y_train_fac, cv=cv, scoring=scorers)

# Friendman Test and poshoc nemenyi

# Accuracy

In [28]:
friedman_result = friedmanchisquare(KNN_cv_results['test_accuracy'], GNB_cv_results['test_accuracy'], PW_cv_results['test_accuracy'], LR_cv_results['test_accuracy'])

print('Friedman test result:', friedman_result)

Friedman test result: FriedmanchisquareResult(statistic=825.633036935705, pvalue=1.1937754551395415e-178)


In [29]:
data = np.array([KNN_cv_results['test_accuracy'], GNB_cv_results['test_accuracy'], PW_cv_results['test_accuracy'], LR_cv_results['test_accuracy']])
    
# Nemenyi test
nemenyi_result = sp.posthoc_nemenyi_friedman(data.T)

ranks = np.argsort(np.argsort(-data, axis=0), axis=0) + 1
mean_ranks = np.mean(ranks, axis=1)
model_names = ['KNN', 'GNB', 'PW', 'LR']
sorted_indices = np.argsort(mean_ranks)
for i in sorted_indices:
    print(f'{model_names[i]}: Rank médio = {mean_ranks[i]:.5f}')

KNN: Rank médio = 1.47333
LR: Rank médio = 1.54000
GNB: Rank médio = 2.98667
PW: Rank médio = 4.00000


# Recall

In [30]:
friedman_result = friedmanchisquare(KNN_cv_results['test_recall_macro'], GNB_cv_results['test_recall_macro'], PW_cv_results['test_recall_macro'], LR_cv_results['test_recall_macro'])

print('Friedman test result:', friedman_result)

Friedman test result: FriedmanchisquareResult(statistic=822.098979591837, pvalue=6.972752068695824e-178)


In [31]:
data = np.array([KNN_cv_results['test_recall_macro'], GNB_cv_results['test_recall_macro'], PW_cv_results['test_recall_macro'], LR_cv_results['test_recall_macro']])
    
# Nemenyi test
nemenyi_result = sp.posthoc_nemenyi_friedman(data.T)

ranks = np.argsort(np.argsort(-data, axis=0), axis=0) + 1
mean_ranks = np.mean(ranks, axis=1)
model_names = ['KNN', 'GNB', 'PW', 'LR']
sorted_indices = np.argsort(mean_ranks)
for i in sorted_indices:
    print(f'{model_names[i]}: Rank médio = {mean_ranks[i]:.5f}')

KNN: Rank médio = 1.49667
LR: Rank médio = 1.51333
GNB: Rank médio = 2.99000
PW: Rank médio = 4.00000


# F1

In [32]:
friedman_result = friedmanchisquare(KNN_cv_results['test_f1_macro'], GNB_cv_results['test_f1_macro'], PW_cv_results['test_f1_macro'], LR_cv_results['test_f1_macro'])

print('Friedman test result:', friedman_result)

Friedman test result: FriedmanchisquareResult(statistic=810.0793438232344, pvalue=2.819970088832914e-175)


In [33]:
data = np.array([KNN_cv_results['test_f1_macro'], GNB_cv_results['test_f1_macro'], PW_cv_results['test_f1_macro'], LR_cv_results['test_f1_macro']])
    
# Nemenyi test
nemenyi_result = sp.posthoc_nemenyi_friedman(data.T)

ranks = np.argsort(np.argsort(-data, axis=0), axis=0) + 1
mean_ranks = np.mean(ranks, axis=1)
model_names = ['KNN', 'GNB', 'PW', 'LR']
sorted_indices = np.argsort(mean_ranks)
for i in sorted_indices:
    print(f'{model_names[i]}: Rank médio = {mean_ranks[i]:.5f}')

LR: Rank médio = 1.42667
KNN: Rank médio = 1.58667
GNB: Rank médio = 2.98667
PW: Rank médio = 4.00000


# Precision

In [34]:
friedman_result = friedmanchisquare(KNN_cv_results['test_precision_macro'], GNB_cv_results['test_precision_macro'], PW_cv_results['test_precision_macro'], LR_cv_results['test_precision_macro'])

print('Friedman test result:', friedman_result)

Friedman test result: FriedmanchisquareResult(statistic=806.6354481369592, pvalue=1.5745414619800617e-174)


In [35]:
data = np.array([KNN_cv_results['test_precision_macro'], GNB_cv_results['test_precision_macro'], PW_cv_results['test_precision_macro'], LR_cv_results['test_precision_macro']])
    
# Nemenyi test
nemenyi_result = sp.posthoc_nemenyi_friedman(data.T)

ranks = np.argsort(np.argsort(-data, axis=0), axis=0) + 1
mean_ranks = np.mean(ranks, axis=1)
model_names = ['KNN', 'GNB', 'PW', 'LR']
sorted_indices = np.argsort(mean_ranks)
for i in sorted_indices:
    print(f'{model_names[i]}: Rank médio = {mean_ranks[i]:.5f}')

LR: Rank médio = 1.47667
KNN: Rank médio = 1.54333
GNB: Rank médio = 2.98000
PW: Rank médio = 4.00000


# Dataset mfeat-fou

In [36]:
# data split
X_train_fou, X_test_fou, y_train_fou, y_test_fou = train_test_split(df_fou,
                                                                    df_target,
                                                                    test_size=0.3,          # Using 30% of the dataset for test
                                                                    stratify= df_target,    # Stratified sample
                                                                    random_state=10)
# Standard Scaler
scaler = StandardScaler()
scaler.fit(X_train_fou)

X_train_fou_scaled = scaler.transform(X_train_fou)
X_test_fou_scaled = scaler.transform(X_test_fou)
# Before fitting the model, convert y_train_fou to a 1D array
y_train_fou = y_train_fou.values.ravel()
y_test_fou = y_test_fou.values.ravel()


In [37]:
# (30x10) stratified fold
cv = RepeatedStratifiedKFold(n_splits = 10, n_repeats = 30, random_state = 10)

In [38]:
# Classifier KNN
knn = neighbors.KNeighborsClassifier(weights='distance', metric='euclidean')
param_grid_KNN = param_grid = {
    'n_neighbors': range(1, 20)
}
grid_KNN_fou = GridSearchCV(knn, param_grid = param_grid_KNN, cv=cv, refit='f1_macro')         
grid_KNN_fou.fit(X_train_fou_scaled, y_train_fou)                                                                                           
best_KNN_fou = grid_KNN_fou.best_params_                                                                                  
print("Best fou-KNN parameter:", best_KNN_fou)                                                                           
best_clf_KNN_fou = KNeighborsClassifier(n_neighbors = best_KNN_fou.get('n_neighbors'), weights = 'distance', metric = 'euclidean')
best_clf_KNN_fou.fit(X_train_fou_scaled, y_train_fou)

Best fou-KNN parameter: {'n_neighbors': 4}


KNeighborsClassifier(metric='euclidean', n_neighbors=4, weights='distance')

In [39]:
# Classifier Gaussian Bayesian
clf_GNB_fou = GaussianNB()   # there are no hyperparameters
clf_GNB_fou.fit(X_train_fou_scaled, y_train_fou) 

GaussianNB()

In [40]:
# Classifier Parzen Window
param_grid_parzen_window = {'bandwidth': np.linspace(0.01, 1, 20)}

parzen_window = KernelDensity(kernel='gaussian')
grid_parzen_window = GridSearchCV(parzen_window, param_grid_parzen_window, cv=cv, refit='f1_macro')
grid_parzen_window.fit(X_train_fou_scaled, y_train_fou)
best_parzen_window = grid_parzen_window.best_params_
print("Best fou parameter:", best_parzen_window)
best_clf_parzen_window_fou = KernelDensity(kernel='gaussian', bandwidth = best_parzen_window.get('bandwidth'))
best_clf_parzen_window_fou.fit(X_train_fou_scaled, y_train_fou)

class Parzen_Window(BaseEstimator, ClassifierMixin):
    def __init__(self, bandwidth=1.0):
        self.bandwidth = bandwidth
        self.kde = None
        self.classes_ = None

    def fit(self, X, y):
        self.classes_ = np.unique(y)
        self.kde = {cls: KernelDensity(bandwidth=self.bandwidth).fit(X[y == cls]) for cls in self.classes_}
        return self

    def predict(self, X):
        log_probs = np.array([self.kde[cls].score_samples(X) for cls in self.classes_]).T
        return self.classes_[np.argmax(log_probs, axis=1)]

best_clf_parzen_window_fou = Parzen_Window(bandwidth=best_parzen_window.get('bandwidth'))
best_clf_parzen_window_fou.fit(X_train_fou_scaled, y_train_fou)

Best fou parameter: {'bandwidth': 0.4789473684210527}


Parzen_Window(bandwidth=0.4789473684210527)

In [41]:
# Logistic Regression
param_grid_LR = {'C':  np.linspace(0.01, 10, 20), 'random_state': np.arange(0, 100, 5)}
grid_LR_fou = GridSearchCV(LogisticRegression(max_iter=1000), param_grid = param_grid_LR, cv=cv, refit='f1_macro')                                                    
grid_LR_fou.fit(X_train_fou_scaled, y_train_fou)                                                                                              
best_LR_fou = grid_LR_fou.best_params_
print("Best fou-LR parameter:", best_LR_fou)
best_clf_LR_fou = LogisticRegression(C = best_LR_fou.get('C'), random_state = best_LR_fou.get('random_state'), max_iter = 1000)
best_clf_LR_fou.fit(X_train_fou_scaled, y_train_fou)

Best fou-LR parameter: {'C': 0.5357894736842106, 'random_state': 0}


LogisticRegression(C=0.5357894736842106, max_iter=1000, random_state=0)

In [42]:
# Voting classifier (majority rule)
classifiers = [
    ('knn', best_clf_KNN_fou),
    ('gnb', clf_GNB_fou),
    ('parzen', best_clf_parzen_window_fou),
    ('lr', best_clf_LR_fou)
]
voting_clf = VotingClassifier(estimators=classifiers, voting='hard')
voting_clf.fit(X_train_fou_scaled, y_train_fou)

# accuracy on train test
train_accuracy = voting_clf.score(X_train_fou_scaled, y_train_fou)
print("Acurácia no conjunto de treinamento:", train_accuracy)

# accuracy on test set
test_accuracy = voting_clf.score(X_test_fou_scaled, y_test_fou)
print("Acurácia no conjunto de teste:", test_accuracy)


Acurácia no conjunto de treinamento: 0.9842857142857143
Acurácia no conjunto de teste: 0.81


# Alternativa B
Obtenha uma estimativa pontual e um intervalo de confiança para cada metrica
de avaliação do classificador (Taxa de erro, precisão, cobertura, F-measure);

In [43]:
y_pred = voting_clf.predict(X_test_fou_scaled)

error_rate = 1 - voting_clf.score(X_test_fou_scaled, y_test_fou)
print("error rate:", error_rate)

coverage_rate = recall_score(y_test_fou, y_pred, average='macro')
print("Recall:", coverage_rate)

precision = precision_score(y_test_fou, y_pred, average='macro')
print("Precision:", precision)

f1 = f1_score(y_test_fou, y_pred, average='macro')
print("F1-score:", f1)

error rate: 0.18999999999999995
Recall: 0.8099999999999999
Precision: 0.8149359435427547
F1-score: 0.8095964709481397


In [44]:
from sklearn.utils import resample

# Function to calculate each metric
def calculate_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred, average='macro')
    precision = precision_score(y_true, y_pred, average='macro')
    f1 = f1_score(y_true, y_pred, average='macro')
    error_rate = 1 - accuracy
    return accuracy, recall, precision, f1, error_rate

# bootstrap iterations number
n_iterations = 1000
# array for result storage
metrics_results = np.zeros((n_iterations, 5))

# running the bootstrap
for i in range(n_iterations):
    # Search for a sample with reposition
    X_resampled, y_resampled = resample(X_train_fou_scaled, y_train_fou)
    # training the model with the sample
    voting_clf.fit(X_resampled, y_resampled)
    # making predictions
    y_pred_resampled = voting_clf.predict(X_resampled)
    # calculating metrics
    metrics_results[i] = calculate_metrics(y_resampled, y_pred_resampled)

# calculating the confidence interval for each metric
alpha = 0.95
lower_p = ((1.0 - alpha) / 2.0) * 100
upper_p = (alpha + ((1.0 - alpha) / 2.0)) * 100
intervals = np.percentile(metrics_results, [lower_p, upper_p], axis=0)

# Imprimindo os intervalos de confiança
metrics_names = ['accuracy', 'Recall', 'Precision', 'F1-score', 'error-rate']
for metric_index, metric_name in enumerate(metrics_names):
    print(f"{metric_name} Confidence interval {alpha*100}%: Lower_p: {intervals[0][metric_index]:.2f}%, Upper_p: {intervals[1][metric_index]:.2f}%")


accuracy Confidence interval 95.0%: Lower_p: 0.98%, Upper_p: 1.00%
Recall Confidence interval 95.0%: Lower_p: 0.98%, Upper_p: 1.00%
Precision Confidence interval 95.0%: Lower_p: 0.99%, Upper_p: 1.00%
F1-score Confidence interval 95.0%: Lower_p: 0.99%, Upper_p: 1.00%
error-rate Confidence interval 95.0%: Lower_p: 0.00%, Upper_p: 0.02%


In [45]:
scorers = {
    'accuracy': make_scorer(accuracy_score),
    'recall_macro': make_scorer(recall_score, average='macro'),
    'f1_macro': make_scorer(f1_score, average='macro'),
    'precision_macro': make_scorer(precision_score, average='macro', zero_division = 0),
}

In [46]:
KNN_cv_results = cross_validate(best_clf_KNN_fou, X_train_fou_scaled, y_train_fou, cv=cv, scoring=scorers)

In [47]:
GNB_cv_results = cross_validate(clf_GNB_fou, X_train_fou_scaled, y_train_fou, cv=cv, scoring=scorers)

In [48]:
PW_cv_results = cross_validate(best_clf_parzen_window_fou, X_train_fou_scaled, y_train_fou, cv=cv, scoring=scorers)

In [49]:
LR_cv_results = cross_validate(best_clf_LR_fou, X_train_fou_scaled, y_train_fou, cv=cv, scoring=scorers)

# Friendman Test and poshoc nemenyi

# Accuracy

In [50]:
friedman_result = friedmanchisquare(KNN_cv_results['test_accuracy'], GNB_cv_results['test_accuracy'], PW_cv_results['test_accuracy'], LR_cv_results['test_accuracy'])

print('Friedman test result:', friedman_result)

Friedman test result: FriedmanchisquareResult(statistic=737.9725981055484, pvalue=1.2241479462477883e-159)


In [51]:
data = np.array([KNN_cv_results['test_accuracy'], GNB_cv_results['test_accuracy'], PW_cv_results['test_accuracy'], LR_cv_results['test_accuracy']])
    
# Nemenyi test
nemenyi_result = sp.posthoc_nemenyi_friedman(data.T)

ranks = np.argsort(np.argsort(-data, axis=0), axis=0) + 1
mean_ranks = np.mean(ranks, axis=1)
model_names = ['KNN', 'GNB', 'PW', 'LR']
sorted_indices = np.argsort(mean_ranks)
for i in sorted_indices:
    print(f'{model_names[i]}: Rank médio = {mean_ranks[i]:.5f}')

LR: Rank médio = 1.57000
KNN: Rank médio = 1.60000
GNB: Rank médio = 2.83000
PW: Rank médio = 4.00000


# Recall

In [52]:
friedman_result = friedmanchisquare(KNN_cv_results['test_recall_macro'], GNB_cv_results['test_recall_macro'], PW_cv_results['test_recall_macro'], LR_cv_results['test_recall_macro'])

print('Friedman test result:', friedman_result)

Friedman test result: FriedmanchisquareResult(statistic=729.145210084034, pvalue=1.004779702038121e-157)


In [53]:
data = np.array([KNN_cv_results['test_recall_macro'], GNB_cv_results['test_recall_macro'], PW_cv_results['test_recall_macro'], LR_cv_results['test_recall_macro']])
    
# Nemenyi test
nemenyi_result = sp.posthoc_nemenyi_friedman(data.T)

ranks = np.argsort(np.argsort(-data, axis=0), axis=0) + 1
mean_ranks = np.mean(ranks, axis=1)
model_names = ['KNN', 'GNB', 'PW', 'LR']
sorted_indices = np.argsort(mean_ranks)
for i in sorted_indices:
    print(f'{model_names[i]}: Rank médio = {mean_ranks[i]:.5f}')

LR: Rank médio = 1.56333
KNN: Rank médio = 1.62000
GNB: Rank médio = 2.81667
PW: Rank médio = 4.00000


# F1

In [54]:
friedman_result = friedmanchisquare(KNN_cv_results['test_f1_macro'], GNB_cv_results['test_f1_macro'], PW_cv_results['test_f1_macro'], LR_cv_results['test_f1_macro'])

print('Friedman test result:', friedman_result)

Friedman test result: FriedmanchisquareResult(statistic=725.2359999999999, pvalue=7.075887149942511e-157)


In [55]:
data = np.array([KNN_cv_results['test_f1_macro'], GNB_cv_results['test_f1_macro'], PW_cv_results['test_f1_macro'], LR_cv_results['test_f1_macro']])
    
# Nemenyi test
nemenyi_result = sp.posthoc_nemenyi_friedman(data.T)

ranks = np.argsort(np.argsort(-data, axis=0), axis=0) + 1
mean_ranks = np.mean(ranks, axis=1)
model_names = ['KNN', 'GNB', 'PW', 'LR']
sorted_indices = np.argsort(mean_ranks)
for i in sorted_indices:
    print(f'{model_names[i]}: Rank médio = {mean_ranks[i]:.5f}')

LR: Rank médio = 1.51000
KNN: Rank médio = 1.66667
GNB: Rank médio = 2.82333
PW: Rank médio = 4.00000


# Precision

In [56]:
friedman_result = friedmanchisquare(KNN_cv_results['test_precision_macro'], GNB_cv_results['test_precision_macro'], PW_cv_results['test_precision_macro'], LR_cv_results['test_precision_macro'])

print('Friedman test result:', friedman_result)

Friedman test result: FriedmanchisquareResult(statistic=689.268, pvalue=4.457777200508526e-149)


In [57]:
data = np.array([KNN_cv_results['test_precision_macro'], GNB_cv_results['test_precision_macro'], PW_cv_results['test_precision_macro'], LR_cv_results['test_precision_macro']])
    
# Nemenyi test
nemenyi_result = sp.posthoc_nemenyi_friedman(data.T)

ranks = np.argsort(np.argsort(-data, axis=0), axis=0) + 1
mean_ranks = np.mean(ranks, axis=1)
model_names = ['KNN', 'GNB', 'PW', 'LR']
sorted_indices = np.argsort(mean_ranks)
for i in sorted_indices:
    print(f'{model_names[i]}: Rank médio = {mean_ranks[i]:.5f}')

LR: Rank médio = 1.61333
KNN: Rank médio = 1.64333
GNB: Rank médio = 2.74333
PW: Rank médio = 4.00000


# Dataset mfeat-zer

In [58]:
# data split
X_train_zer, X_test_zer, y_train_zer, y_test_zer = train_test_split(df_zer,
                                                                    df_target,
                                                                    test_size=0.3,          # Using 30% of the dataset for test
                                                                    stratify= df_target,    # Stratified sample
                                                                    random_state=10)
# Standard Scaler
scaler = StandardScaler()
scaler.fit(X_train_zer)

X_train_zer_scaled = scaler.transform(X_train_zer)
X_test_zer_scaled = scaler.transform(X_test_zer)
# Before fitting the model, convert y_train_zer to a 1D array
y_train_zer = y_train_zer.values.ravel()
y_test_zer = y_test_zer.values.ravel()


In [59]:
# (30x10) stratified fold
cv = RepeatedStratifiedKFold(n_splits = 10, n_repeats = 30, random_state = 10)

In [60]:
# Classifier KNN
knn = neighbors.KNeighborsClassifier(weights='distance', metric='euclidean')
param_grid_KNN = param_grid = {
    'n_neighbors': range(1, 20)
}
grid_KNN_zer = GridSearchCV(knn, param_grid = param_grid_KNN, cv=cv, refit='f1_macro')         
grid_KNN_zer.fit(X_train_zer_scaled, y_train_zer)                                                                                           
best_KNN_zer = grid_KNN_zer.best_params_                                                                                  
print("Best zer-KNN parameter:", best_KNN_zer)                                                                           
best_clf_KNN_zer = KNeighborsClassifier(n_neighbors = best_KNN_zer.get('n_neighbors'), weights = 'distance', metric = 'euclidean')
best_clf_KNN_zer.fit(X_train_zer_scaled, y_train_zer)

Best zer-KNN parameter: {'n_neighbors': 4}


KNeighborsClassifier(metric='euclidean', n_neighbors=4, weights='distance')

In [61]:
# Classifier Gaussian Bayesian
clf_GNB_zer = GaussianNB()   # there are no hyperparameters
clf_GNB_zer.fit(X_train_zer_scaled, y_train_zer) 

GaussianNB()

In [62]:
# Classifier Parzen Window
param_grid_parzen_window = {'bandwidth': np.linspace(0.01, 1, 20)}

parzen_window = KernelDensity(kernel='gaussian')
grid_parzen_window = GridSearchCV(parzen_window, param_grid_parzen_window, cv=cv, refit='f1_macro')
grid_parzen_window.fit(X_train_zer_scaled, y_train_zer)
best_parzen_window = grid_parzen_window.best_params_
print("Best zer parameter:", best_parzen_window)
best_clf_parzen_window_zer = KernelDensity(kernel='gaussian', bandwidth = best_parzen_window.get('bandwidth'))
best_clf_parzen_window_zer.fit(X_train_zer_scaled, y_train_zer)

class Parzen_Window(BaseEstimator, ClassifierMixin):
    def __init__(self, bandwidth=1.0):
        self.bandwidth = bandwidth
        self.kde = None
        self.classes_ = None

    def fit(self, X, y):
        self.classes_ = np.unique(y)
        self.kde = {cls: KernelDensity(bandwidth=self.bandwidth).fit(X[y == cls]) for cls in self.classes_}
        return self

    def predict(self, X):
        log_probs = np.array([self.kde[cls].score_samples(X) for cls in self.classes_]).T
        return self.classes_[np.argmax(log_probs, axis=1)]

best_clf_parzen_window_zer = Parzen_Window(bandwidth=best_parzen_window.get('bandwidth'))
best_clf_parzen_window_zer.fit(X_train_zer_scaled, y_train_zer)

Best zer parameter: {'bandwidth': 0.37473684210526315}


Parzen_Window(bandwidth=0.37473684210526315)

In [63]:
# Logistic Regression
param_grid_LR = {'C':  np.linspace(0.01, 10, 20), 'random_state': np.arange(0, 100, 5)}
grid_LR_zer = GridSearchCV(LogisticRegression(max_iter=1000), param_grid = param_grid_LR, cv=cv, refit='f1_macro')                                                    
grid_LR_zer.fit(X_train_zer_scaled, y_train_zer)                                                                                              
best_LR_zer = grid_LR_zer.best_params_
print("Best zer-LR parameter:", best_LR_zer)
best_clf_LR_zer = LogisticRegression(C = best_LR_zer.get('C'), random_state = best_LR_zer.get('random_state'), max_iter = 1000)
best_clf_LR_zer.fit(X_train_zer_scaled, y_train_zer)

Best zer-LR parameter: {'C': 1.5873684210526318, 'random_state': 0}


LogisticRegression(C=1.5873684210526318, max_iter=1000, random_state=0)

In [64]:
# Voting classifier (majority rule)
classifiers = [
    ('knn', best_clf_KNN_zer),
    ('gnb', clf_GNB_zer),
    ('parzen', best_clf_parzen_window_zer),
    ('lr', best_clf_LR_zer)
]
voting_clf = VotingClassifier(estimators=classifiers, voting='hard')
voting_clf.fit(X_train_zer_scaled, y_train_zer)

# accuracy on train test
train_accuracy = voting_clf.score(X_train_zer_scaled, y_train_zer)
print("Acurácia no conjunto de treinamento:", train_accuracy)

# accuracy on test set
test_accuracy = voting_clf.score(X_test_zer_scaled, y_test_zer)
print("Acurácia no conjunto de teste:", test_accuracy)


Acurácia no conjunto de treinamento: 0.9671428571428572
Acurácia no conjunto de teste: 0.8133333333333334


# Alternativa B
Obtenha uma estimativa pontual e um intervalo de confiança para cada metrica
de avaliação do classificador (Taxa de erro, precisão, cobertura, F-measure);

In [65]:
y_pred = voting_clf.predict(X_test_zer_scaled)

error_rate = 1 - voting_clf.score(X_test_zer_scaled, y_test_zer)
print("error rate:", error_rate)

coverage_rate = recall_score(y_test_zer, y_pred, average='macro')
print("Recall:", coverage_rate)

precision = precision_score(y_test_zer, y_pred, average='macro')
print("Precision:", precision)

f1 = f1_score(y_test_zer, y_pred, average='macro')
print("F1-score:", f1)

error rate: 0.18666666666666665
Recall: 0.8133333333333332
Precision: 0.805957560862051
F1-score: 0.8087098259024346


In [66]:
from sklearn.utils import resample

# Function to calculate each metric
def calculate_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred, average='macro')
    precision = precision_score(y_true, y_pred, average='macro')
    f1 = f1_score(y_true, y_pred, average='macro')
    error_rate = 1 - accuracy
    return accuracy, recall, precision, f1, error_rate

# bootstrap iterations number
n_iterations = 1000
# array for result storage
metrics_results = np.zeros((n_iterations, 5))

# running the bootstrap
for i in range(n_iterations):
    # Search for a sample with reposition
    X_resampled, y_resampled = resample(X_train_zer_scaled, y_train_zer)
    # training the model with the sample
    voting_clf.fit(X_resampled, y_resampled)
    # making predictions
    y_pred_resampled = voting_clf.predict(X_resampled)
    # calculating metrics
    metrics_results[i] = calculate_metrics(y_resampled, y_pred_resampled)

# calculating the confidence interval for each metric
alpha = 0.95
lower_p = ((1.0 - alpha) / 2.0) * 100
upper_p = (alpha + ((1.0 - alpha) / 2.0)) * 100
intervals = np.percentile(metrics_results, [lower_p, upper_p], axis=0)

# Imprimindo os intervalos de confiança
metrics_names = ['accuracy', 'Recall', 'Precision', 'F1-score', 'error-rate']
for metric_index, metric_name in enumerate(metrics_names):
    print(f"{metric_name} Confidence interval {alpha*100}%: Lower_p: {intervals[0][metric_index]:.2f}%, Upper_p: {intervals[1][metric_index]:.2f}%")


accuracy Confidence interval 95.0%: Lower_p: 0.95%, Upper_p: 0.98%
Recall Confidence interval 95.0%: Lower_p: 0.95%, Upper_p: 0.98%
Precision Confidence interval 95.0%: Lower_p: 0.96%, Upper_p: 0.98%
F1-score Confidence interval 95.0%: Lower_p: 0.95%, Upper_p: 0.98%
error-rate Confidence interval 95.0%: Lower_p: 0.02%, Upper_p: 0.05%


In [67]:
scorers = {
    'accuracy': make_scorer(accuracy_score),
    'recall_macro': make_scorer(recall_score, average='macro'),
    'f1_macro': make_scorer(f1_score, average='macro'),
    'precision_macro': make_scorer(precision_score, average='macro', zero_division = 0),
}

In [68]:
KNN_cv_results = cross_validate(best_clf_KNN_zer, X_train_zer_scaled, y_train_zer, cv=cv, scoring=scorers)

In [69]:
GNB_cv_results = cross_validate(clf_GNB_zer, X_train_zer_scaled, y_train_zer, cv=cv, scoring=scorers)

In [70]:
PW_cv_results = cross_validate(best_clf_parzen_window_zer, X_train_zer_scaled, y_train_zer, cv=cv, scoring=scorers)

In [71]:
LR_cv_results = cross_validate(best_clf_LR_zer, X_train_zer_scaled, y_train_zer, cv=cv, scoring=scorers)

# Friendman Test and poshoc nemenyi

# Accuracy

In [72]:
friedman_result = friedmanchisquare(KNN_cv_results['test_accuracy'], GNB_cv_results['test_accuracy'], PW_cv_results['test_accuracy'], LR_cv_results['test_accuracy'])

print('Friedman test result:', friedman_result)

Friedman test result: FriedmanchisquareResult(statistic=829.1313301823092, pvalue=2.0806251274283883e-179)


In [73]:
data = np.array([KNN_cv_results['test_accuracy'], GNB_cv_results['test_accuracy'], PW_cv_results['test_accuracy'], LR_cv_results['test_accuracy']])
    
# Nemenyi test
nemenyi_result = sp.posthoc_nemenyi_friedman(data.T)

ranks = np.argsort(np.argsort(-data, axis=0), axis=0) + 1
mean_ranks = np.mean(ranks, axis=1)
model_names = ['KNN', 'GNB', 'PW', 'LR']
sorted_indices = np.argsort(mean_ranks)
for i in sorted_indices:
    print(f'{model_names[i]}: Rank médio = {mean_ranks[i]:.5f}')

LR: Rank médio = 1.37333
KNN: Rank médio = 1.63333
GNB: Rank médio = 2.99333
PW: Rank médio = 4.00000


# Recall

In [74]:
friedman_result = friedmanchisquare(KNN_cv_results['test_recall_macro'], GNB_cv_results['test_recall_macro'], PW_cv_results['test_recall_macro'], LR_cv_results['test_recall_macro'])

print('Friedman test result:', friedman_result)

Friedman test result: FriedmanchisquareResult(statistic=823.6173212487411, pvalue=3.2666321529191915e-178)


In [75]:
data = np.array([KNN_cv_results['test_recall_macro'], GNB_cv_results['test_recall_macro'], PW_cv_results['test_recall_macro'], LR_cv_results['test_recall_macro']])
    
# Nemenyi test
nemenyi_result = sp.posthoc_nemenyi_friedman(data.T)

ranks = np.argsort(np.argsort(-data, axis=0), axis=0) + 1
mean_ranks = np.mean(ranks, axis=1)
model_names = ['KNN', 'GNB', 'PW', 'LR']
sorted_indices = np.argsort(mean_ranks)
for i in sorted_indices:
    print(f'{model_names[i]}: Rank médio = {mean_ranks[i]:.5f}')

LR: Rank médio = 1.34667
KNN: Rank médio = 1.66333
GNB: Rank médio = 2.99000
PW: Rank médio = 4.00000


# F1

In [76]:
friedman_result = friedmanchisquare(KNN_cv_results['test_f1_macro'], GNB_cv_results['test_f1_macro'], PW_cv_results['test_f1_macro'], LR_cv_results['test_f1_macro'])

print('Friedman test result:', friedman_result)

Friedman test result: FriedmanchisquareResult(statistic=814.8280000000004, pvalue=2.632406745080964e-176)


In [77]:
data = np.array([KNN_cv_results['test_f1_macro'], GNB_cv_results['test_f1_macro'], PW_cv_results['test_f1_macro'], LR_cv_results['test_f1_macro']])
    
# Nemenyi test
nemenyi_result = sp.posthoc_nemenyi_friedman(data.T)

ranks = np.argsort(np.argsort(-data, axis=0), axis=0) + 1
mean_ranks = np.mean(ranks, axis=1)
model_names = ['KNN', 'GNB', 'PW', 'LR']
sorted_indices = np.argsort(mean_ranks)
for i in sorted_indices:
    print(f'{model_names[i]}: Rank médio = {mean_ranks[i]:.5f}')

LR: Rank médio = 1.33667
KNN: Rank médio = 1.67333
GNB: Rank médio = 2.99000
PW: Rank médio = 4.00000


# Precision

In [78]:
friedman_result = friedmanchisquare(KNN_cv_results['test_precision_macro'], GNB_cv_results['test_precision_macro'], PW_cv_results['test_precision_macro'], LR_cv_results['test_precision_macro'])

print('Friedman test result:', friedman_result)

Friedman test result: FriedmanchisquareResult(statistic=809.9040000000005, pvalue=3.0780313419824725e-175)


In [79]:
data = np.array([KNN_cv_results['test_precision_macro'], GNB_cv_results['test_precision_macro'], PW_cv_results['test_precision_macro'], LR_cv_results['test_precision_macro']])
    
# Nemenyi test
nemenyi_result = sp.posthoc_nemenyi_friedman(data.T)

ranks = np.argsort(np.argsort(-data, axis=0), axis=0) + 1
mean_ranks = np.mean(ranks, axis=1)
model_names = ['KNN', 'GNB', 'PW', 'LR']
sorted_indices = np.argsort(mean_ranks)
for i in sorted_indices:
    print(f'{model_names[i]}: Rank médio = {mean_ranks[i]:.5f}')

LR: Rank médio = 1.36667
KNN: Rank médio = 1.64667
GNB: Rank médio = 2.98667
PW: Rank médio = 4.00000
